In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import datetime

In [2]:
# 우리나라 기후 불러오기

In [3]:
df = pd.read_csv('./국내기후(백령도제외).csv', index_col=0)
# df

In [4]:
df['시간'] = pd.to_datetime(df['시간'], format='%Y-%m-%d')

In [5]:
# df[df['시간'].dt.year > 2006]
df_k = df[df['시간'] > '2006-12-09']
df_k = df_k.reset_index()
df_k.drop('index', axis=1, inplace=True)

In [6]:
df_k

,시간,관측지점,평균 기온,최저 기온,최고 기온,1시간 최다강수량,일강수량,최대 순간풍속,최대 순간 풍속 풍향,최대 풍속,...,합계 일사,일 최심신적설,일 최심적설,합계 3시간 신적설,평균 전운량,평균 중하층운량,평균 지면온도,합계 대형증발량,합계 소형증발량,안개 계속 시간
0,2006-12-10,서울,-0.4,-3.5,3.7,0.0,0.0,6.5,290.0,4.1,...,9.51,0.0,0.0,0.0,2.5,0.5,1.0,2.721902,1.7,0.0
1,2006-12-11,서울,1.8,-2.0,6.0,0.0,0.0,5.0,70.0,3.2,...,6.24,0.0,0.0,0.0,4.9,1.5,1.2,2.721902,1.0,0.0
2,2006-12-12,서울,5.2,1.2,9.5,0.0,0.0,5.1,250.0,3.3,...,5.64,0.0,0.0,0.0,6.1,4.3,3.4,2.721902,1.0,0.0
3,2006-12-13,서울,5.7,3.5,9.8,0.0,0.0,8.6,320.0,5.0,...,4.63,0.0,0.0,0.0,5.5,3.9,4.3,2.721902,0.8,0.0
4,2006-12-14,서울,3.3,1.1,6.1,0.0,0.0,6.0,360.0,3.6,...,4.76,0.0,0.0,0.0,4.1,2.9,1.8,2.721902,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44003,2021-12-27,제주,4.1,0.8,6.5,0.0,0.1,12.8,20.0,6.8,...,1.37,0.5,1.2,0.5,8.1,8.1,3.9,1.000000,1.4,0.0
44004,2021-12-28,제주,7.2,5.9,8.4,0.0,0.0,9.1,360.0,5.3,...,0.91,0.0,0.0,0.0,8.3,8.0,5.6,0.900000,1.3,0.0
44005,2021-12-29,제주,9.3,6.6,11.5,0.0,0.4,12.8,340.0,7.9,...,4.94,0.0,0.0,0.0,8.5,7.8,6.9,2.300000,3.3,0.0
44006,2021-12-30,제주,6.9,5.8,8.4,0.0,0.0,18.1,340.0,10.1,...,4.85,0.0,0.0,0.0,8.0,8.0,5.9,2.300000,3.2,0.0


In [7]:
df_seoul = df_k[df_k['시간'] < '2021-09-01']

In [8]:
df_seoul.columns
df_seoul.drop(['1시간 최다강수량','1시간 최다 일사량', '합계 일조 시간','합계 일사','일 최심신적설','일 최심적설',
               '합계 3시간 신적설',
             '합계 대형증발량','합계 소형증발량'], axis=1, inplace=True)

In [9]:
len(df_seoul.columns)

24

In [10]:
df_seoul.rename(columns={'시간':'일시', '관측지점':'도시명'}, inplace=True)

In [11]:
df_seoul['도시명'].unique()

array(['서울', '강릉', '춘천', '청주', '전주', '광주', '대구', '제주'], dtype=object)

In [12]:
west_w = ['서울','전주','광주']
east_w = ['강릉','춘천','청주','대구']
south_w = ['제주']

In [13]:
no_li_w=[]
for i in df_seoul['도시명']:
    if i in west_w:
        no_li_w.append('ko_w')
    elif i in east_w:
        no_li_w.append('ko_e')
    else:
        no_li_w.append('ko_s')
        
# no_li_w

In [14]:
df_seoul['군집'] = no_li_w

In [15]:
len(df_seoul.columns)

25

In [16]:
df_seoul_n = df_seoul.groupby(['군집', '일시']).agg({'평균 기온':'mean','최저 기온':'min','최고 기온':'max',
                                                '일강수량':'mean','최대 순간풍속': 'mean','최대 순간 풍속 풍향':'mean',
                                               '최대 풍속':'mean','최대 풍속 풍향':'mean','평균 풍속':'mean','풍정합':'mean',
                                                '최대 풍향':'mean',
                                               '평균 이슬점온도':'mean','평균 상대습도':'mean','평균 증기압':'mean',
                                               '평균 현지기압':'mean','최고 해면 기압':'max','최저 해면기압':'min',
                                               '평균 해면기압':'mean',
                                               '평균 전운량':'mean','평균 중하층운량':'mean','평균 지면온도':'mean',
                                               '안개 계속 시간':'mean'})
df_seoul_n = df_seoul_n.reset_index()

In [17]:
df_seoul_n

,군집,일시,평균 기온,최저 기온,최고 기온,일강수량,최대 순간풍속,최대 순간 풍속 풍향,최대 풍속,최대 풍속 풍향,...,평균 상대습도,평균 증기압,평균 현지기압,최고 해면 기압,최저 해면기압,평균 해면기압,평균 전운량,평균 중하층운량,평균 지면온도,안개 계속 시간
0,ko_e,2006-12-10,1.550000,-4.1,8.1,0.000000,6.550000,267.500000,3.975,277.500000,...,51.875000,3.325000,1020.275000,1029.9,1023.5,1027.150000,2.125000,0.350000,1.850000,0.000000
1,ko_e,2006-12-11,1.675000,-6.9,9.2,0.000000,4.975000,197.500000,3.225,220.000000,...,58.175000,3.900000,1020.225000,1030.7,1023.7,1027.125000,4.375000,0.925000,1.725000,0.000000
2,ko_e,2006-12-12,4.675000,-3.3,13.1,0.000000,6.875000,235.000000,3.975,247.500000,...,61.850000,5.100000,1018.025000,1027.3,1021.3,1024.750000,3.475000,1.725000,3.125000,0.000000
3,ko_e,2006-12-13,5.725000,-2.0,11.5,0.000000,7.500000,272.500000,3.875,267.500000,...,67.275000,6.050000,1015.500000,1025.4,1018.1,1022.225000,6.650000,3.725000,4.100000,1.500000
4,ko_e,2006-12-14,4.050000,-2.8,9.1,0.000000,6.125000,280.000000,4.025,277.500000,...,60.100000,4.750000,1014.225000,1024.0,1017.3,1020.975000,5.075000,2.250000,3.325000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16132,ko_w,2021-08-27,22.533333,19.9,28.2,22.933333,8.233333,256.666667,5.000,263.333333,...,92.233333,25.266667,1004.433333,1014.4,1010.3,1012.766667,8.766667,5.566667,24.333333,0.166667
16133,ko_w,2021-08-28,24.100000,19.7,30.4,0.066667,6.000000,220.000000,3.200,213.333333,...,82.166667,24.300000,1005.633333,1015.5,1012.3,1013.866667,7.200000,4.833333,25.933333,0.000000
16134,ko_w,2021-08-29,23.966667,19.4,27.9,20.100000,6.766667,296.666667,4.000,286.666667,...,89.300000,26.600000,1005.133333,1015.5,1012.0,1013.366667,8.866667,5.433333,26.066667,0.000000
16135,ko_w,2021-08-30,24.600000,21.2,29.9,0.633333,5.400000,293.333333,2.800,280.000000,...,83.700000,25.700000,1007.200000,1016.8,1013.7,1015.433333,8.433333,5.000000,26.100000,0.000000


In [18]:
tdf_k = pd.DataFrame()

gun_k_li = list(df_seoul_n['군집'].unique())

for gun in gun_k_li:
    tmp_k = df_seoul_n[df_seoul_n['군집']==gun].drop(['군집','일시'],axis=1).reset_index(drop=True)
    tmp_k.columns = gun+tmp_k.columns
    tdf_k = pd.concat([tdf_k,tmp_k],axis=1)
df_k_g = tdf_k
df_k_g

,ko_e평균 기온,ko_e최저 기온,ko_e최고 기온,ko_e일강수량,ko_e최대 순간풍속,ko_e최대 순간 풍속 풍향,ko_e최대 풍속,ko_e최대 풍속 풍향,ko_e평균 풍속,ko_e풍정합,...,ko_w평균 상대습도,ko_w평균 증기압,ko_w평균 현지기압,ko_w최고 해면 기압,ko_w최저 해면기압,ko_w평균 해면기압,ko_w평균 전운량,ko_w평균 중하층운량,ko_w평균 지면온도,ko_w안개 계속 시간
0,1.550,-4.1,8.1,0.000,6.550,267.5,3.975,277.5,1.950,1684.50,...,59.200000,3.866667,1019.333333,1030.0,1025.9,1028.333333,2.366667,0.400000,2.333333,0.000000
1,1.675,-6.9,9.2,0.000,4.975,197.5,3.225,220.0,1.425,1231.00,...,58.566667,4.266667,1018.000000,1030.0,1024.1,1026.900000,4.666667,1.366667,2.866667,0.000000
2,4.675,-3.3,13.1,0.000,6.875,235.0,3.975,247.5,1.575,1360.75,...,65.466667,6.000000,1016.666667,1027.2,1023.0,1025.466667,3.266667,1.966667,4.233333,0.000000
3,5.725,-2.0,11.5,0.000,7.500,272.5,3.875,267.5,1.725,1490.25,...,75.233333,7.533333,1014.533333,1025.6,1020.8,1023.300000,6.933333,4.666667,6.000000,0.000000
4,4.050,-2.8,9.1,0.000,6.125,280.0,4.025,277.5,1.475,1274.25,...,66.333333,5.666667,1012.933333,1024.3,1019.4,1021.766667,5.700000,3.266667,3.900000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5374,23.200,19.8,28.7,8.875,6.350,280.0,3.900,297.5,1.075,945.00,...,92.233333,25.266667,1004.433333,1014.4,1010.3,1012.766667,8.766667,5.566667,24.333333,0.166667
5375,24.575,19.3,31.9,0.075,5.525,192.5,3.350,192.5,1.450,1249.50,...,82.166667,24.300000,1005.633333,1015.5,1012.3,1013.866667,7.200000,4.833333,25.933333,0.000000
5376,23.675,18.6,30.6,6.600,5.725,225.0,3.475,220.0,1.400,1191.00,...,89.300000,26.600000,1005.133333,1015.5,1012.0,1013.366667,8.866667,5.433333,26.066667,0.000000
5377,24.200,19.2,30.6,0.050,4.500,165.0,2.750,217.5,1.250,1054.00,...,83.700000,25.700000,1007.200000,1016.8,1013.7,1015.433333,8.433333,5.000000,26.100000,0.000000


In [18]:
# tdf_k = pd.DataFrame({'일시':df_seoul_n['일시'].unique()})

# gun_k_li = list(df_seoul_n['군집'].unique())

# for gun in gun_k_li:
#     tmp_k = df_seoul_n[df_seoul_n['군집']==gun].drop('군집',axis=1)
#     tmp_k.columns = ['일시', f'{gun}평균 기온', f'{gun}최저 기온', f'{gun}최고 기온', f'{gun}1시간 최다강수량', f'{gun}일강수량', 
#                      f'{gun}최대 순간풍속',f'{gun}최대 순간 풍속 풍향',f'{gun}최대 풍속',f'{gun}최대 풍속 풍향'
#                      f'{gun}평균 풍속',
#                      f'{gun}풍정합',f'{gun}최대 풍향',f'{gun}평균 이슬점온도',f'{gun}평균 상대습도',f'{gun}평균 증기압',
#                      f'{gun}평균 현지기압',f'{gun}최고 해면 기압',f'{gun}최저 해면기압',f'{gun}평균 해면기압',f'{gun}합계 일조 시간',
#                      f'{gun}1시간 최다 일사량',f'{gun}합계 일사',f'{gun}일 최심신적설',f'{gun}일 최심적설',f'{gun}합계 3시간 신적설',
#                      f'{gun}평균 전운량',f'{gun}평균 중하층운량',f'{gun}평균 지면온도',f'{gun}합계 대형증발량',f'{gun}합계 소형증발량',
#                      f'{gun}안개 계속 시간']
#     tdf_k = pd.merge(tdf_k,tmp_k, how='left',on='일시')
# df_k_g = tdf_k
# df_k_g

In [19]:
df_seoul_n.columns

Index(['군집', '일시', '평균 기온', '최저 기온', '최고 기온', '일강수량', '최대 순간풍속', '최대 순간 풍속 풍향',
       '최대 풍속', '최대 풍속 풍향', '평균 풍속', '풍정합', '최대 풍향', '평균 이슬점온도', '평균 상대습도',
       '평균 증기압', '평균 현지기압', '최고 해면 기압', '최저 해면기압', '평균 해면기압', '평균 전운량',
       '평균 중하층운량', '평균 지면온도', '안개 계속 시간'],
      dtype='object')

In [20]:
# 우리나라 고층기후 가져오기

In [21]:
df_k_h = pd.read_csv('./preprocessed_고층기후데이터.csv', index_col=0)
# df_k_h

In [22]:
df_k_h = df_k_h.reset_index()

In [23]:
df_k_h.columns

Index(['일시', '50hPa_고도', '50hPa_기온', '50hPa_이슬점', '50hPa_풍향', '50hPa_풍속',
       '70hPa_고도', '70hPa_기온', '70hPa_이슬점', '70hPa_풍향', '70hPa_풍속',
       '100hPa_고도', '100hPa_기온', '100hPa_이슬점', '100hPa_풍향', '100hPa_풍속',
       '150hPa_고도', '150hPa_기온', '150hPa_이슬점', '150hPa_풍향', '150hPa_풍속',
       '200hPa_고도', '200hPa_기온', '200hPa_이슬점', '200hPa_풍향', '200hPa_풍속',
       '250hPa_고도', '250hPa_기온', '250hPa_이슬점', '250hPa_풍향', '250hPa_풍속',
       '300hPa_고도', '300hPa_기온', '300hPa_이슬점', '300hPa_풍향', '300hPa_풍속',
       '400hPa_고도', '400hPa_기온', '400hPa_이슬점', '400hPa_풍향', '400hPa_풍속',
       '500hPa_고도', '500hPa_기온', '500hPa_이슬점', '500hPa_풍향', '500hPa_풍속',
       '700hPa_고도', '700hPa_기온', '700hPa_이슬점', '700hPa_풍향', '700hPa_풍속',
       '850hPa_고도', '850hPa_기온', '850hPa_이슬점', '850hPa_풍향', '850hPa_풍속',
       '925hPa_고도', '925hPa_기온', '925hPa_이슬점', '925hPa_풍향', '925hPa_풍속',
       '1000hPa_고도', '1000hPa_기온', '1000hPa_이슬점', '1000hPa_풍향', '1000hPa_풍속'],
      dtype='object')

In [24]:
df_k_h['일시'] = pd.to_datetime(df_k_h['일시'], format='%Y-%m-%d')

In [25]:
df_k_h = df_k_h[df_k_h['일시'] > '2006-12-09']
df_k_h = df_k_h[df_k_h['일시'] < '2021-09-01']
df_k_h = df_k_h.reset_index()
df_k_h.drop('index', axis=1, inplace=True)

In [26]:
df_k_h

,일시,50hPa_고도,50hPa_기온,50hPa_이슬점,50hPa_풍향,50hPa_풍속,70hPa_고도,70hPa_기온,70hPa_이슬점,70hPa_풍향,...,925hPa_고도,925hPa_기온,925hPa_이슬점,925hPa_풍향,925hPa_풍속,1000hPa_고도,1000hPa_기온,1000hPa_이슬점,1000hPa_풍향,1000hPa_풍속
0,2006-12-10,20405.000000,-57.500,-67.250000,258.750000,53.000000,18300.000000,-58.650000,-68.150000,262.500000,...,839.00,-4.100000,-20.600000,237.500000,10.000000,220.500000,-0.600,-12.600000,261.25,5.750000
1,2006-12-11,20377.500000,-57.325,-69.075000,260.000000,47.500000,18280.000000,-60.475000,-71.850000,258.750000,...,829.25,-1.700000,-9.500000,153.750000,11.000000,205.500000,1.625,-8.625000,117.50,5.750000
2,2006-12-12,20452.500000,-59.150,-73.150000,260.000000,37.250000,18342.500000,-60.550000,-74.300000,260.000000,...,826.00,1.750000,-3.300000,266.250000,12.000000,193.250000,5.700,-0.050000,157.50,5.000000
3,2006-12-13,20492.500000,-58.575,-78.450000,256.666667,38.666667,18385.000000,-63.050000,-80.050000,260.000000,...,811.00,1.375000,-6.825000,296.250000,11.250000,178.250000,5.800,-1.600000,270.00,7.750000
4,2006-12-14,20477.500000,-58.600,-72.350000,258.333333,50.583333,18390.000000,-63.650000,-76.400000,258.125000,...,793.25,-0.775000,-5.525000,288.750000,11.500000,166.000000,3.600,-5.300000,247.50,4.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5374,2021-08-27,20969.583333,-57.950,-70.825000,67.500000,9.708333,18878.750000,-65.283333,-73.658333,259.583333,...,790.50,17.600000,15.700000,338.750000,12.750000,117.750000,21.050,20.000000,232.50,4.250000
5375,2021-08-28,20968.750000,-58.750,-74.125000,124.375000,11.375000,18865.000000,-64.400000,-74.150000,247.500000,...,803.50,18.150000,14.775000,302.500000,8.000000,128.500000,21.300,18.250000,203.75,5.750000
5376,2021-08-29,20958.750000,-58.200,-81.200000,113.125000,6.875000,18861.250000,-64.600000,-79.850000,260.000000,...,806.00,18.200000,15.025000,178.750000,8.250000,131.250000,21.850,19.800000,163.75,3.750000
5377,2021-08-30,20970.000000,-57.825,-74.700000,142.500000,9.625000,18868.750000,-63.175000,-73.925000,251.250000,...,821.50,19.750000,13.500000,261.250000,4.500000,144.500000,22.650,19.875000,258.75,3.750000


In [ ]:
# 주변국 데이터 불러오기

In [27]:
df_w = pd.read_csv('./preprocessed_주변국 기상데이터_199601to202108.csv', index_col=0)
# df_w.drop('index', axis=1, inplace=True)

df_w['일시'] = pd.to_datetime(df_w['일시'], format='%Y-%m-%d')
df_w = df_w[df_w['일시'] > '2006-12-09']
df_w = df_w.reset_index()
df_w.drop('index', axis=1, inplace=True)

df_w

,도시명,일시,평균기온,최고기온,최저기온,습도,풍향,풍속,강수량,해면기압,이슬점 온도,군집
0,청진,2006-12-10,-2.371429,0.4,-5.0,54.857143,40.0,1.428571,0.000000,1027.042857,-10.800000,n2
1,청진,2006-12-11,-2.062500,2.1,-4.9,84.250000,290.0,0.375000,0.000000,1027.775000,-4.375000,n2
2,청진,2006-12-12,0.600000,5.9,-2.3,79.000000,90.0,0.142857,0.000000,1022.028571,-2.828571,n2
3,청진,2006-12-13,2.466667,4.7,-0.8,48.666667,320.0,3.333333,0.000000,1019.133333,-9.900000,n2
4,청진,2006-12-14,0.362500,3.1,-2.7,59.250000,320.0,0.875000,2.123669,1021.525000,-8.062500,n2
...,...,...,...,...,...,...,...,...,...,...,...,...
155986,후쿠오카,2021-08-27,29.175000,32.8,25.6,70.495833,30.0,5.125000,0.000000,1014.391667,23.162500,n5
155987,후쿠오카,2021-08-28,28.641667,32.3,25.1,72.362500,40.0,4.958333,0.000000,1014.325000,23.000000,n5
155988,후쿠오카,2021-08-29,28.908333,32.4,25.5,72.687500,290.0,5.041667,0.000000,1014.754167,23.375000,n5
155989,후쿠오카,2021-08-30,28.904167,32.5,25.8,75.795833,310.0,4.541667,0.000000,1015.587500,24.112500,n5


In [28]:
df_ww = df_w.groupby(['군집', '일시']).agg({ '습도':'mean','풍향':'mean', 
                                            '풍속':'mean', '강수량':'mean','해면기압':'mean', '이슬점 온도':'mean'})
df_ww = df_ww.reset_index()
# df_ww

In [29]:
tdf = pd.DataFrame({'일시':df_ww['일시'].unique()})

In [30]:
gunli = list(df_ww['군집'].unique())

for gun in gunli:
    tmp = df_ww[df_ww['군집']==gun].drop('군집',axis=1)
    tmp.columns = ['일시', f'{gun}습도', f'{gun}풍향', f'{gun}풍속', f'{gun}강수량', f'{gun}해면기압', f'{gun}이슬점 온도']
    tdf = pd.merge(tdf,tmp, how='left',on='일시')
df_w_w = tdf
df_w_w

,일시,n1습도,n1풍향,n1풍속,n1강수량,n1해면기압,n1이슬점 온도,n2습도,n2풍향,n2풍속,...,n6풍속,n6강수량,n6해면기압,n6이슬점 온도,n7습도,n7풍향,n7풍속,n7강수량,n7해면기압,n7이슬점 온도
0,2006-12-10,56.928571,230.000000,1.160714,0.000000,1028.351488,-9.167857,69.571429,27.5,0.607143,...,2.017857,0.000000,1025.631786,-7.886429,64.904167,110.000000,2.041667,0.000000,1028.425000,-1.791667
1,2006-12-11,65.468750,202.500000,2.062500,0.445600,1026.965625,-4.284375,77.772321,80.0,0.281250,...,3.010714,0.000000,1021.521429,-3.608929,76.262500,263.333333,2.500000,-0.047822,1024.695833,1.304167
2,2006-12-12,72.770833,179.274917,1.005952,0.000000,1024.439583,-0.572024,73.654762,40.0,0.285714,...,2.364286,0.000000,1023.791071,-4.638214,79.154167,323.333333,1.791667,0.133333,1025.512500,3.220833
3,2006-12-13,62.916667,248.750000,1.416667,0.000000,1023.255208,-3.912500,55.000000,102.5,1.750000,...,2.282143,0.000000,1024.350714,-11.458214,71.641667,220.000000,1.666667,1.000000,1025.920833,1.837500
4,2006-12-14,60.915179,167.500000,0.805804,0.000000,1021.968527,-6.006473,66.916667,115.0,0.531250,...,2.389286,0.000000,1019.422857,-9.447500,68.450000,170.000000,1.875000,0.000000,1022.854167,-0.191667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5374,2021-08-27,82.468750,111.250000,0.906250,0.880792,1011.759375,20.028125,80.593750,350.0,0.937500,...,2.800000,0.000000,1009.500000,16.523214,78.830556,240.000000,2.125000,-0.047822,1012.870833,21.366667
5375,2021-08-28,77.921875,115.000000,1.500000,0.000000,1013.007812,19.660938,75.125000,195.0,1.375000,...,1.996429,-0.594099,1011.968571,16.480000,81.301190,106.666667,1.404762,-0.739609,1012.662500,22.993452
5376,2021-08-29,83.125000,162.500000,1.031250,1.669771,1013.484375,19.915625,77.875000,42.5,1.218750,...,1.982143,-0.915458,1013.549643,17.045000,87.795833,210.000000,2.083333,-0.702848,1013.058333,23.450000
5377,2021-08-30,77.578125,183.750000,0.937500,0.000000,1015.429688,19.173437,73.281250,157.5,1.406250,...,1.510714,-0.915262,1015.932500,17.566429,83.704167,220.000000,1.916667,-0.625120,1012.554167,24.179167


In [ ]:
# 황사데이터 불러오기

In [31]:
df_h = pd.read_csv('./황사.csv', encoding='euckr')
# df_w.drop('index', axis=1, inplace=True)
df_h

,지점,지점명,일시,일 미세먼지 농도(㎍/㎥)
0,90,속초,2006-12-10,22
1,90,속초,2006-12-11,26
2,90,속초,2006-12-12,47
3,90,속초,2006-12-13,84
4,90,속초,2006-12-14,50
...,...,...,...,...
151309,501,연평도,2022-02-27,34
151310,501,연평도,2022-02-28,28
151311,501,연평도,2022-03-01,26
151312,501,연평도,2022-03-02,23


In [32]:
df_h['지점명'].unique()

array(['속초', '북춘천', '광덕산', '대관령', '춘천', '백령도', '서울', '울릉도독도(감)', '관악산',
       '수원', '영월', '울진', '안면도(감)', '추풍령', '안동', '군산', '대구', '오성산', '전주',
       '울산', '광주', '구덕산', '흑산도', '진도(첨찰산)', '고산', '진주', '강화', '북격렬비도',
       '천안', '진도군', '문경', '영덕', '연평도'], dtype=object)

In [33]:
df_h['일시'] = pd.to_datetime(df_h['일시'], format='%Y-%m-%d')
df_h = df_h[df_h['일시'] > '2006-12-09']
df_h = df_h[df_h['일시'] < '2021-09-01']
df_h = df_h.reset_index()
df_h.drop('index', axis=1, inplace=True)

In [34]:
df_h['일시'].sort_values

<bound method Series.sort_values of 0        2006-12-10
1        2006-12-11
2        2006-12-12
3        2006-12-13
4        2006-12-14
            ...    
133473   2021-08-27
133474   2021-08-28
133475   2021-08-29
133476   2021-08-30
133477   2021-08-31
Name: 일시, Length: 133478, dtype: datetime64[ns]>

In [35]:
df_h

,지점,지점명,일시,일 미세먼지 농도(㎍/㎥)
0,90,속초,2006-12-10,22
1,90,속초,2006-12-11,26
2,90,속초,2006-12-12,47
3,90,속초,2006-12-13,84
4,90,속초,2006-12-14,50
...,...,...,...,...
133473,501,연평도,2021-08-27,20
133474,501,연평도,2021-08-28,26
133475,501,연평도,2021-08-29,16
133476,501,연평도,2021-08-30,34


In [36]:
# 서
west_li = ['서울', '강화', '수원', '안면도(감)', '군산', '광주', '광덕산', '백령도', '관악산', '오성산', 
           '전주', '북격렬비도', '천안', '연평도']

In [37]:
# 동
east_li = ['춘천', '추풍령', '안동', '대구', '속초', '북춘천', '대관령', '춘천', '울릉도독도(감)', '영월', '울진', '문경', '영덕']

In [38]:
# 남
south_li = ['고산', '진도군', '구덕산', '울산', '흑산도', '진도(첨찰산)', '진주']

In [39]:
df_h['지점명'].tail(50)

133428    연평도
133429    연평도
133430    연평도
133431    연평도
133432    연평도
133433    연평도
133434    연평도
133435    연평도
133436    연평도
133437    연평도
133438    연평도
133439    연평도
133440    연평도
133441    연평도
133442    연평도
133443    연평도
133444    연평도
133445    연평도
133446    연평도
133447    연평도
133448    연평도
133449    연평도
133450    연평도
133451    연평도
133452    연평도
133453    연평도
133454    연평도
133455    연평도
133456    연평도
133457    연평도
133458    연평도
133459    연평도
133460    연평도
133461    연평도
133462    연평도
133463    연평도
133464    연평도
133465    연평도
133466    연평도
133467    연평도
133468    연평도
133469    연평도
133470    연평도
133471    연평도
133472    연평도
133473    연평도
133474    연평도
133475    연평도
133476    연평도
133477    연평도
Name: 지점명, dtype: object

In [40]:
no_li=[]
for i in df_h['지점명']:
    if i in west_li:
        no_li.append('d1')
    elif i in east_li:
        no_li.append('d2')
    else:
        no_li.append('d3')
        
# no_li

In [41]:
df_h['군집'] = no_li

In [42]:
df_h.isna().sum()

지점                0
지점명               0
일시                0
일 미세먼지 농도(㎍/㎥)    0
군집                0
dtype: int64

In [43]:
df_h['군집'].value_counts()

d1    59597
d2    45849
d3    28032
Name: 군집, dtype: int64

In [44]:
df_h.drop('지점', axis=1, inplace=True)
df_h.rename({'일 미세먼지 농도(㎍/㎥)' : '농도'}, axis=1, inplace=True)

In [45]:
df_h_test = df_h

In [46]:
df_h_test = df_h_test.groupby(['군집', '일시']).agg({ '농도':'mean'})
df_h_test = df_h_test.reset_index()

In [47]:
df_h_test[df_h_test['농도'].isna()]

,군집,일시,농도


In [48]:
df_h_test

,군집,일시,농도
0,d1,2006-12-10,24.555556
1,d1,2006-12-11,38.111111
2,d1,2006-12-12,63.444444
3,d1,2006-12-13,95.777778
4,d1,2006-12-14,64.666667
...,...,...,...
16120,d3,2021-08-27,22.166667
16121,d3,2021-08-28,16.250000
16122,d3,2021-08-29,19.250000
16123,d3,2021-08-30,19.400000


In [49]:
df_h_test[df_h_test['농도'].isna()]

,군집,일시,농도


In [50]:
tdf_h = pd.DataFrame({'일시':df_h_test['일시'].unique()})

In [51]:
gun_li_h = list(df_h_test['군집'].unique())

for gun in gun_li_h:
    tmp_h = df_h_test[df_h_test['군집']==gun].drop('군집',axis=1)
    tmp_h.columns = ['일시', f'{gun}농도']
    tdf_h = pd.merge(tdf_h,tmp_h, how='left',on='일시')
df_h_n = tdf_h
df_h_n

,일시,d1농도,d2농도,d3농도
0,2006-12-10,24.555556,21.000000,23.000000
1,2006-12-11,38.111111,20.600000,17.333333
2,2006-12-12,63.444444,42.166667,26.666667
3,2006-12-13,95.777778,78.500000,43.333333
4,2006-12-14,64.666667,70.833333,60.000000
...,...,...,...,...
5372,2021-08-27,29.000000,32.200000,22.166667
5373,2021-08-28,30.272727,22.900000,16.250000
5374,2021-08-29,24.555556,23.200000,19.250000
5375,2021-08-30,35.363636,27.200000,19.400000


In [52]:
import datetime as dt

dat_a = pd.DataFrame({'일시':np.arange(dt.datetime(2006,12,10),dt.datetime(2021,9, 1),dt.timedelta(days=1))})
dat_a['일시'] = dat_a['일시'].apply(lambda x : x.strftime('%Y-%m-%d'))

In [53]:
dat_a['일시'] = dat_a['일시'].astype('datetime64')

In [54]:
dat_a

,일시
0,2006-12-10
1,2006-12-11
2,2006-12-12
3,2006-12-13
4,2006-12-14
...,...
5374,2021-08-27
5375,2021-08-28
5376,2021-08-29
5377,2021-08-30


In [55]:
df_h_test = pd.merge(dat_a, df_h_n, how='left', on='일시')

In [56]:
df_h_test

,일시,d1농도,d2농도,d3농도
0,2006-12-10,24.555556,21.000000,23.000000
1,2006-12-11,38.111111,20.600000,17.333333
2,2006-12-12,63.444444,42.166667,26.666667
3,2006-12-13,95.777778,78.500000,43.333333
4,2006-12-14,64.666667,70.833333,60.000000
...,...,...,...,...
5374,2021-08-27,29.000000,32.200000,22.166667
5375,2021-08-28,30.272727,22.900000,16.250000
5376,2021-08-29,24.555556,23.200000,19.250000
5377,2021-08-30,35.363636,27.200000,19.400000


In [57]:
df_h_test[df_h_test['d1농도'].isna()]

,일시,d1농도,d2농도,d3농도
4685,2019-10-08,NaN,NaN,NaN
4686,2019-10-09,NaN,NaN,NaN


In [58]:
df_h_test['d1농도'].interpolate(inplace=True)
df_h_test['d2농도'].interpolate(inplace=True)
df_h_test['d3농도'].interpolate(inplace=True)

In [59]:
df_h_test.isna().sum()

일시      0
d1농도    0
d2농도    0
d3농도    0
dtype: int64

In [60]:
df_h_test

,일시,d1농도,d2농도,d3농도
0,2006-12-10,24.555556,21.000000,23.000000
1,2006-12-11,38.111111,20.600000,17.333333
2,2006-12-12,63.444444,42.166667,26.666667
3,2006-12-13,95.777778,78.500000,43.333333
4,2006-12-14,64.666667,70.833333,60.000000
...,...,...,...,...
5374,2021-08-27,29.000000,32.200000,22.166667
5375,2021-08-28,30.272727,22.900000,16.250000
5376,2021-08-29,24.555556,23.200000,19.250000
5377,2021-08-30,35.363636,27.200000,19.400000


In [133]:
df_h_test_a = df_h_test

In [134]:
df_h_test_a

,일시,d1농도,d2농도,d3농도,check,target
0,2006-12-10,24.555556,21.000000,23.000000,0,1
1,2006-12-11,38.111111,20.600000,17.333333,0,0
2,2006-12-12,63.444444,42.166667,26.666667,0,1
3,2006-12-13,95.777778,78.500000,43.333333,1,0
4,2006-12-14,64.666667,70.833333,60.000000,0,0
...,...,...,...,...,...,...
5374,2021-08-27,29.000000,32.200000,22.166667,0,0
5375,2021-08-28,30.272727,22.900000,16.250000,0,0
5376,2021-08-29,24.555556,23.200000,19.250000,0,0
5377,2021-08-30,35.363636,27.200000,19.400000,0,0


In [135]:
check_li = [] 
for h in range(len(df_h_test_a)):
    if (df_h_test_a.iloc[h,1] > 80):
        check_li.append(1)
    elif (df_h_test_a.iloc[h,2] > 80):
        check_li.append(1)
    elif (df_h_test_a.iloc[h,3] > 80):
        check_li.append(1)
    else:
        check_li.append(0)
check_li

[0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [136]:
len(check_li)

5379

In [137]:
df_h_test_a['check'] = check_li

In [138]:
df_h_test_a['target'] = df_h_test_a.check.shift(-3)
df_h_test_a

,일시,d1농도,d2농도,d3농도,check,target
0,2006-12-10,24.555556,21.000000,23.000000,0,1.0
1,2006-12-11,38.111111,20.600000,17.333333,0,0.0
2,2006-12-12,63.444444,42.166667,26.666667,0,1.0
3,2006-12-13,95.777778,78.500000,43.333333,1,0.0
4,2006-12-14,64.666667,70.833333,60.000000,0,0.0
...,...,...,...,...,...,...
5374,2021-08-27,29.000000,32.200000,22.166667,0,0.0
5375,2021-08-28,30.272727,22.900000,16.250000,0,0.0
5376,2021-08-29,24.555556,23.200000,19.250000,0,NaN
5377,2021-08-30,35.363636,27.200000,19.400000,0,NaN


In [139]:
df_h_test_a['target'].fillna(0, inplace=True)
df_h_test_a['target'] = df_h_test_a['target'].astype(int)

In [142]:
df_h_test_aa = df_h_test_a
df_h_test_aa.drop(['d1농도','d2농도','d3농도'], axis=1, inplace=True)
df_h_test_aa

,일시,check,target
0,2006-12-10,0,1
1,2006-12-11,0,0
2,2006-12-12,0,1
3,2006-12-13,1,0
4,2006-12-14,0,0
...,...,...,...
5374,2021-08-27,0,0
5375,2021-08-28,0,0
5376,2021-08-29,0,0
5377,2021-08-30,0,0


In [110]:
df_h_test_w = df_h_test[['일시','d1농도']]

In [111]:
df_h_test_w['check'] = df_h_test_w['d1농도'].apply(lambda x: 1 if x > 80 else 0)
df_h_test_w

,일시,d1농도,check
0,2006-12-10,24.555556,0
1,2006-12-11,38.111111,0
2,2006-12-12,63.444444,0
3,2006-12-13,95.777778,1
4,2006-12-14,64.666667,0
...,...,...,...
5374,2021-08-27,29.000000,0
5375,2021-08-28,30.272727,0
5376,2021-08-29,24.555556,0
5377,2021-08-30,35.363636,0


In [112]:
df_h_test_w['target'] = df_h_test_w.check.shift(-2)

In [113]:
df_h_test_w

,일시,d1농도,check,target
0,2006-12-10,24.555556,0,0.0
1,2006-12-11,38.111111,0,1.0
2,2006-12-12,63.444444,0,0.0
3,2006-12-13,95.777778,1,1.0
4,2006-12-14,64.666667,0,0.0
...,...,...,...,...
5374,2021-08-27,29.000000,0,0.0
5375,2021-08-28,30.272727,0,0.0
5376,2021-08-29,24.555556,0,0.0
5377,2021-08-30,35.363636,0,NaN


In [114]:
df_h_test_w['target'].fillna(0, inplace=True)

In [115]:
df_h_test_w['target'] = df_h_test_w['target'].astype(int)

In [116]:
df_h_test_w

,일시,d1농도,check,target
0,2006-12-10,24.555556,0,0
1,2006-12-11,38.111111,0,1
2,2006-12-12,63.444444,0,0
3,2006-12-13,95.777778,1,1
4,2006-12-14,64.666667,0,0
...,...,...,...,...
5374,2021-08-27,29.000000,0,0
5375,2021-08-28,30.272727,0,0
5376,2021-08-29,24.555556,0,0
5377,2021-08-30,35.363636,0,0


In [121]:
df_h_test_w_a = df_h_test_a['target']

In [122]:
df_h_test_w_a

0       1
1       0
2       1
3       0
4       0
       ..
5374    0
5375    0
5376    0
5377    0
5378    0
Name: target, Length: 5379, dtype: int32

In [ ]:
#  df_k_g - 한국
# df_k_h - 고층
# df_w_w - 세계기후
# df_h_test - 황사

In [ ]:
# merge

In [65]:
df_m = pd.concat([df_k_h,df_k_g], axis=1)
# pd.concat([tdf_k,tmp_k],axis=1)


In [66]:
df_m.isna().sum().sum()

0

In [518]:
#1 우리나라+고층+주변국+황사서쪽
df_m = pd.concat([df_k_h,df_k_g], axis=1)
df_m = pd.merge(df_m,df_w_w, how='left', on='일시')
df_ml_m = pd.merge(df_ml,df_h_test_w, how='left', on='일시')

df_a = df_ml_m[df_ml_m['일시'] < '2019-01-01']
df_b = df_ml_m[df_ml_m['일시'] > '2019-12-31']

In [81]:
# 2 우리나라+고층+주변국+황사서쪽
df_mm = pd.concat([df_w_w,df_k_g ], axis=1)
df_mm_m = pd.merge(df_mm,df_h_test_w, how='left', on='일시')

df_a = df_mm_m[df_mm_m['일시'] < '2019-01-01']
df_b = df_mm_m[df_mm_m['일시'] > '2019-12-31']

In [82]:
# 3 우리나라+고층+주변국+황사
df_mm = pd.concat([df_w_w,df_k_g ], axis=1)
df_mm_mm = pd.merge(df_mm,df_h_test_a, how='left', on='일시')

df_a = df_mm_mm[df_mm_mm['일시'] < '2019-01-01']
df_b = df_mm_mm[df_mm_mm['일시'] > '2019-12-31']

In [149]:
# 4 우리나라+고층+주변국+황사(정답값 빼기?)
df_mm = pd.concat([df_w_w,df_k_g ], axis=1)
df_mm_mm = pd.merge(df_mm,df_h_test_aa, how='left', on='일시')

df_a = df_mm_mm[df_mm_mm['일시'] < '2019-01-01']
df_b = df_mm_mm[df_mm_mm['일시'] > '2019-12-31']

In [151]:
df_ml_m = pd.concat([df_a,df_b])

In [152]:
df_ml_m

,일시,n1습도,n1풍향,n1풍속,n1강수량,n1해면기압,n1이슬점 온도,n2습도,n2풍향,n2풍속,...,ko_w평균 현지기압,ko_w최고 해면 기압,ko_w최저 해면기압,ko_w평균 해면기압,ko_w평균 전운량,ko_w평균 중하층운량,ko_w평균 지면온도,ko_w안개 계속 시간,check,target
0,2006-12-10,56.928571,230.000000,1.160714,0.000000,1028.351488,-9.167857,69.571429,27.5,0.607143,...,1019.333333,1030.0,1025.9,1028.333333,2.366667,0.400000,2.333333,0.000000,0,1
1,2006-12-11,65.468750,202.500000,2.062500,0.445600,1026.965625,-4.284375,77.772321,80.0,0.281250,...,1018.000000,1030.0,1024.1,1026.900000,4.666667,1.366667,2.866667,0.000000,0,0
2,2006-12-12,72.770833,179.274917,1.005952,0.000000,1024.439583,-0.572024,73.654762,40.0,0.285714,...,1016.666667,1027.2,1023.0,1025.466667,3.266667,1.966667,4.233333,0.000000,0,1
3,2006-12-13,62.916667,248.750000,1.416667,0.000000,1023.255208,-3.912500,55.000000,102.5,1.750000,...,1014.533333,1025.6,1020.8,1023.300000,6.933333,4.666667,6.000000,0.000000,1,0
4,2006-12-14,60.915179,167.500000,0.805804,0.000000,1021.968527,-6.006473,66.916667,115.0,0.531250,...,1012.933333,1024.3,1019.4,1021.766667,5.700000,3.266667,3.900000,0.000000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5374,2021-08-27,82.468750,111.250000,0.906250,0.880792,1011.759375,20.028125,80.593750,350.0,0.937500,...,1004.433333,1014.4,1010.3,1012.766667,8.766667,5.566667,24.333333,0.166667,0,0
5375,2021-08-28,77.921875,115.000000,1.500000,0.000000,1013.007812,19.660938,75.125000,195.0,1.375000,...,1005.633333,1015.5,1012.3,1013.866667,7.200000,4.833333,25.933333,0.000000,0,0
5376,2021-08-29,83.125000,162.500000,1.031250,1.669771,1013.484375,19.915625,77.875000,42.5,1.218750,...,1005.133333,1015.5,1012.0,1013.366667,8.866667,5.433333,26.066667,0.000000,0,0
5377,2021-08-30,77.578125,183.750000,0.937500,0.000000,1015.429688,19.173437,73.281250,157.5,1.406250,...,1007.200000,1016.8,1013.7,1015.433333,8.433333,5.000000,26.100000,0.000000,0,0


In [153]:
df_ml_m.columns

Index(['일시', 'n1습도', 'n1풍향', 'n1풍속', 'n1강수량', 'n1해면기압', 'n1이슬점 온도', 'n2습도',
       'n2풍향', 'n2풍속',
       ...
       'ko_w평균 현지기압', 'ko_w최고 해면 기압', 'ko_w최저 해면기압', 'ko_w평균 해면기압',
       'ko_w평균 전운량', 'ko_w평균 중하층운량', 'ko_w평균 지면온도', 'ko_w안개 계속 시간', 'check',
       'target'],
      dtype='object', length=111)

In [ ]:
# 머신러닝

In [154]:
y = df_ml_m['target']
x = df_ml_m.drop(['일시','check','target'], axis=1)

In [155]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

In [156]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier

In [157]:
from sklearn.metrics import classification_report, roc_auc_score

In [158]:
lr = LogisticRegression()
svc = SVC(probability=True)
rfc = RandomForestClassifier()
xg = XGBClassifier()
knn = KNeighborsClassifier()
lgbm = LGBMClassifier()

In [159]:
models = [lr, svc, rfc, xg, knn, lgbm]

In [160]:
for m in models:
    m.fit(x_train, y_train)
    pred = m.predict(x_test)
    preb = m.predict_proba(x_test)
    
    print(classification_report(y_test, pred))
    print('-'*50)
    print(roc_auc_score(y_test, preb[:,1]))
    print('='*100)
    

              precision    recall  f1-score   support

           0       0.93      1.00      0.96       934
           1       0.00      0.00      0.00        69

    accuracy                           0.93      1003
   macro avg       0.47      0.50      0.48      1003
weighted avg       0.87      0.93      0.90      1003

--------------------------------------------------
0.7258635136393259
              precision    recall  f1-score   support

           0       0.93      1.00      0.96       934
           1       0.00      0.00      0.00        69

    accuracy                           0.93      1003
   macro avg       0.47      0.50      0.48      1003
weighted avg       0.87      0.93      0.90      1003

--------------------------------------------------
0.6727803121993606
              precision    recall  f1-score   support

           0       0.93      1.00      0.96       934
           1       0.00      0.00      0.00        69

    accuracy                           0.9

In [161]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [162]:
kfold = KFold()
for m in models:
    score = cross_val_score(m, x_train, y_train, cv=kfold)
    
    print(score)
    print(np.mean(score))
    print(np.std(score))
    print('='*100)

[0.93026152 0.93765586 0.94389027 0.93765586 0.93017456]
0.9359276155812214
0.005188060298301563
[0.93026152 0.93765586 0.94389027 0.93765586 0.93017456]
0.9359276155812214
0.005188060298301563
[0.93026152 0.93765586 0.94389027 0.93765586 0.93017456]
0.9359276155812214
0.005188060298301563
[11:23:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:23:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:23:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/lear

In [163]:
pd.Series(xg.feature_importances_, x.columns).sort_values(ascending=False).head(60)

ko_s최저 기온          0.037423
ko_w평균 현지기압        0.034276
ko_s평균 증기압         0.030063
ko_s안개 계속 시간       0.022045
n3이슬점 온도           0.021467
n5이슬점 온도           0.021128
ko_w평균 해면기압        0.017889
n3강수량              0.015790
ko_s평균 기온          0.014825
ko_s평균 풍속          0.014799
n1이슬점 온도           0.014214
n1해면기압             0.014035
ko_w안개 계속 시간       0.013861
ko_e평균 중하층운량       0.012718
ko_s최대 풍향          0.012249
ko_s최대 풍속 풍향       0.012217
n2풍속               0.011946
n1습도               0.011887
ko_e최대 풍속          0.011593
ko_w최저 기온          0.011395
n6강수량              0.011395
ko_e평균 전운량         0.011376
ko_s평균 이슬점온도       0.011159
n6해면기압             0.011051
n3습도               0.010957
n7풍속               0.010854
ko_s평균 상대습도        0.010819
n7습도               0.010787
ko_w평균 풍속          0.010616
n5강수량              0.010601
ko_e평균 상대습도        0.010525
n3해면기압             0.010284
ko_e최대 순간풍속        0.010215
ko_w평균 전운량         0.010042
ko_w최저 해면기압        0.010026
ko_s최저 해면기압        0

In [164]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()

std.fit(x_train)
x_train_std = std.transform(x_train)
x_test_std = std.transform(x_test)

In [165]:

models = [lr, svc, rfc, xg, knn, lgbm]

for m in models:
    m.fit(x_train_std, y_train)
    pred = m.predict(x_test_std)
    preb = m.predict_proba(x_test_std)
    
    print(classification_report(y_test, pred))
    print('-'*50)
    print(roc_auc_score(y_test, preb[:,1]))
    print('='*100)
    

              precision    recall  f1-score   support

           0       0.93      1.00      0.96       934
           1       0.00      0.00      0.00        69

    accuracy                           0.93      1003
   macro avg       0.47      0.50      0.48      1003
weighted avg       0.87      0.93      0.90      1003

--------------------------------------------------
0.8257145517177171
              precision    recall  f1-score   support

           0       0.93      1.00      0.96       934
           1       0.00      0.00      0.00        69

    accuracy                           0.93      1003
   macro avg       0.47      0.50      0.48      1003
weighted avg       0.87      0.93      0.90      1003

--------------------------------------------------
0.7422182912826242
              precision    recall  f1-score   support

           0       0.93      1.00      0.96       934
           1       0.00      0.00      0.00        69

    accuracy                           0.9

In [166]:
pd.Series(xg.feature_importances_, x.columns).sort_values(ascending=False).head(60)

ko_s최저 기온          0.037423
ko_w평균 현지기압        0.034276
ko_s평균 증기압         0.030063
ko_s안개 계속 시간       0.022045
n3이슬점 온도           0.021467
n5이슬점 온도           0.021128
ko_w평균 해면기압        0.017889
n3강수량              0.015790
ko_s평균 기온          0.014825
ko_s평균 풍속          0.014799
n1이슬점 온도           0.014214
n1해면기압             0.014035
ko_w안개 계속 시간       0.013861
ko_e평균 중하층운량       0.012718
ko_s최대 풍향          0.012249
ko_s최대 풍속 풍향       0.012217
n2풍속               0.011946
n1습도               0.011887
ko_e최대 풍속          0.011593
ko_w최저 기온          0.011395
n6강수량              0.011395
ko_e평균 전운량         0.011376
ko_s평균 이슬점온도       0.011159
n6해면기압             0.011051
n3습도               0.010957
n7풍속               0.010854
ko_s평균 상대습도        0.010819
n7습도               0.010787
ko_w평균 풍속          0.010616
n5강수량              0.010601
ko_e평균 상대습도        0.010525
n3해면기압             0.010284
ko_e최대 순간풍속        0.010215
ko_w평균 전운량         0.010042
ko_w최저 해면기압        0.010026
ko_s최저 해면기압        0

In [167]:
# !pip install imblearn

In [168]:
# !pip install imbalanced-learn

In [169]:
import imblearn

In [170]:
from imblearn.over_sampling import SMOTE

In [171]:
smote = SMOTE(random_state=100)
x_train_std_sm, y_train_sm = smote.fit_resample(x_train_std, y_train)

In [172]:
y_train_sm.value_counts()

0    3754
1    3754
Name: target, dtype: int64

In [173]:
for m in models:
    m.fit(x_train_std_sm, y_train_sm)
    pred = m.predict(x_test_std)
    preb = m.predict_proba(x_test_std)
    
    print(classification_report(y_test, pred))
    print('-'*50)
    print(roc_auc_score(y_test, preb[:,1]))
    print('='*100)
    

              precision    recall  f1-score   support

           0       0.97      0.74      0.84       934
           1       0.17      0.74      0.28        69

    accuracy                           0.74      1003
   macro avg       0.57      0.74      0.56      1003
weighted avg       0.92      0.74      0.80      1003

--------------------------------------------------
0.8347143344815815
              precision    recall  f1-score   support

           0       0.95      0.88      0.92       934
           1       0.19      0.36      0.25        69

    accuracy                           0.85      1003
   macro avg       0.57      0.62      0.58      1003
weighted avg       0.90      0.85      0.87      1003

--------------------------------------------------
0.7633057133103683
              precision    recall  f1-score   support

           0       0.93      0.98      0.96       934
           1       0.24      0.07      0.11        69

    accuracy                           0.9

In [626]:
x

,n1습도,n1풍향,n1풍속,n1강수량,n1해면기압,n1이슬점 온도,n2습도,n2풍향,n2풍속,n2강수량,...,n3평균 증기압,n3평균 현지기압,n3최고 해면 기압,n3최저 해면기압,n3평균 해면기압,n3평균 전운량,n3평균 중하층운량,n3평균 지면온도,n3안개 계속 시간,d1농도
0,56.928571,230.000000,1.160714,0.000000,1028.351488,-9.167857,69.571429,27.5,0.607143,0.000000,...,6.1,1024.8,1028.6,1026.2,1027.4,7.8,6.8,7.7,0.0,24.555556
1,65.468750,202.500000,2.062500,0.445600,1026.965625,-4.284375,77.772321,80.0,0.281250,0.000000,...,7.2,1023.6,1027.6,1024.4,1026.1,8.9,5.9,8.3,0.0,38.111111
2,72.770833,179.274917,1.005952,0.000000,1024.439583,-0.572024,73.654762,40.0,0.285714,0.000000,...,7.9,1022.6,1026.6,1024.0,1025.1,9.5,5.6,10.2,0.0,63.444444
3,62.916667,248.750000,1.416667,0.000000,1023.255208,-3.912500,55.000000,102.5,1.750000,0.000000,...,9.0,1020.7,1025.3,1021.0,1023.2,8.5,6.1,10.1,0.0,95.777778
4,60.915179,167.500000,0.805804,0.000000,1021.968527,-6.006473,66.916667,115.0,0.531250,0.530917,...,7.5,1018.9,1023.8,1018.7,1021.5,7.9,3.0,8.8,0.0,64.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4765,59.703125,218.750000,1.843750,0.000000,1023.760937,-9.764062,67.312500,80.0,1.562500,0.000000,...,5.4,1023.0,1028.1,1021.8,1025.6,8.6,8.6,6.2,0.0,25.230769
4766,73.593750,150.000000,1.765625,0.478187,1026.767187,-4.006250,78.718750,125.0,0.875000,0.275000,...,6.0,1025.7,1030.2,1027.1,1028.3,4.8,3.8,7.2,0.0,35.923077
4767,71.625000,152.098983,1.906250,0.445600,1025.548437,-1.056250,82.500000,177.5,0.812500,0.000000,...,11.6,1020.3,1028.3,1019.5,1022.9,9.6,7.0,9.3,0.0,35.923077
4768,69.750000,295.000000,3.625000,1.598263,1023.862500,-4.415625,76.812500,187.5,1.875000,1.289723,...,10.4,1021.7,1029.8,1020.7,1024.2,8.5,7.1,10.8,0.0,27.307692
